In [24]:
## Underwriting model
# Assumptions
from pandas.core import construction


cap_rate_at_sale = 0.0525
selling_cost = 0.065
acquisition_price = 1420000
down_payment = 1.0
acquisition_cost_pct = 0.04
equity_gp = 1.0
equity_lp = 0.0
aum_fee = 0.02
construction_conversion_sf = 460
construction_conversion_px_sf = 200
construction_detached_sf = 2247
construction_detached_px_sf = 245
construction_carport_sf = 0
construction_carport_px_sf = 100
construction_sitework_sf = 12589
construction_sitework_px_sf = 5
design_permitting_fee_sf = 2707
design_permitting_fee_px_sf = 35
expense_ratio = 0.28
general_vacancy_pct = 0.03
year_complete_construction = 1.5
year_complete_renovation = 1.5
base_rent_adjustment_during_construction = 0.5
year_of_sale = 5


base_rent = 6710
base_rent_growth_rate = 0.03
base_rent_adu = 12250
base_rent_adu_growth_rate = 0.03
new_units_adu = 4

# Calculations

# Calculate construction costs
construction_conversion_cost = construction_conversion_sf * construction_conversion_px_sf
construction_detached_cost = construction_detached_sf * construction_detached_px_sf
construction_carport_cost = construction_carport_sf * construction_carport_px_sf
construction_sitework_cost = construction_sitework_sf * construction_sitework_px_sf
design_permitting_cost = design_permitting_fee_sf * design_permitting_fee_px_sf

total_construction_cost = (construction_conversion_cost + construction_detached_cost + 
                          construction_carport_cost + construction_sitework_cost + 
                          design_permitting_cost)

total_equity_in = total_construction_cost + acquisition_price * (1 + acquisition_cost_pct)


In [18]:
total_construction_cost

800205

In [38]:

# Initialize data structure to store yearly projections
years_data = {}

for year in range(0, 6):
    years_data[year] = {
        'base_rent_house': 0,
        'base_rent_adu': 0,
        'general_vacancy': 0,
        'effective_gross_income': 0,
        'expenses': 0,
        'NOI': 0,
        'property_value': 0,
        'aum_fee': 0,
        'cash_flow_from_operations': 0,
        'proceeds_from_sale_of_property': 0,
        'selling_costs': 0,
        'initial_equity_investment': 0,
        'total_cash_flow_to_equity': 0
    }

    if year == 0:
        years_data[year]['initial_equity_investment'] = total_equity_in
        continue
    if year > year_complete_construction:
        years_data[year]['base_rent_house'] = 12*base_rent*(1+base_rent_growth_rate)**year
    else:
        years_data[year]['base_rent_house'] = 12*base_rent*(1+base_rent_growth_rate)**year*base_rent_adjustment_during_construction
    if year+1 > year_complete_construction:
        years_data[year]['base_rent_adu'] = 12*base_rent_adu*(1+base_rent_adu_growth_rate)**year
    else:
        years_data[year]['base_rent_adu'] = 0

    years_data[year]['general_vacancy'] = general_vacancy_pct*years_data[year]['base_rent_house']
    years_data[year]['effective_gross_income'] = (years_data[year]['base_rent_house'] + years_data[year]['base_rent_adu'] - years_data[year]['general_vacancy'])
    years_data[year]['expenses'] = years_data[year]['effective_gross_income']*expense_ratio
    years_data[year]['NOI'] = years_data[year]['effective_gross_income'] - years_data[year]['expenses']
    years_data[year]['property_value'] = years_data[year]['NOI']/cap_rate_at_sale
    years_data[year]['aum_fee'] = total_equity_in*aum_fee
    years_data[year]['cash_flow_from_operations'] = years_data[year]['NOI'] - years_data[year]['aum_fee']

    if year == year_of_sale:
        years_data[year]['proceeds_from_sale_of_property'] = years_data[year]['property_value']
        years_data[year]['selling_costs'] = years_data[year]['property_value']*selling_cost
    else:
        years_data[year]['proceeds_from_sale_of_property'] = 0
        years_data[year]['selling_costs'] = 0

    years_data[year]['total_cash_flow_to_equity'] = years_data[year]['cash_flow_from_operations'] + years_data[year]['proceeds_from_sale_of_property'] - years_data[year]['selling_costs']


    


In [39]:
import pandas as pd
df = pd.DataFrame(years_data)
pd.set_option('display.float_format', '{:.2f}'.format)
df

,0,1,2,3,4,5
base_rent_house,0.00,41467.80,85423.67,87986.38,90625.97,93344.75
base_rent_adu,0.00,151410.00,155952.30,160630.87,165449.80,170413.29
general_vacancy,0.00,1244.03,2562.71,2639.59,2718.78,2800.34
effective_gross_income,0.00,191633.77,238813.26,245977.66,253356.99,260957.69
expenses,0.00,53657.45,66867.71,68873.74,70939.96,73068.15
NOI,0.00,137976.31,171945.55,177103.91,182417.03,187889.54
property_value,0.00,2628120.22,3275153.25,3373407.85,3474610.09,3578848.39
aum_fee,0.00,45540.10,45540.10,45540.10,45540.10,45540.10
cash_flow_from_operations,0.00,92436.21,126405.45,131563.81,136876.93,142349.44
proceeds_from_sale_of_property,0.00,0.00,0.00,0.00,0.00,3578848.39


In [40]:
df_transposed = df.T
df_transposed


,base_rent_house,base_rent_adu,general_vacancy,effective_gross_income,expenses,NOI,property_value,aum_fee,cash_flow_from_operations,proceeds_from_sale_of_property,selling_costs,initial_equity_investment,total_cash_flow_to_equity
0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2277005.00,0.00
1,41467.80,151410.00,1244.03,191633.77,53657.45,137976.31,2628120.22,45540.10,92436.21,0.00,0.00,0.00,92436.21
2,85423.67,155952.30,2562.71,238813.26,66867.71,171945.55,3275153.25,45540.10,126405.45,0.00,0.00,0.00,126405.45
3,87986.38,160630.87,2639.59,245977.66,68873.74,177103.91,3373407.85,45540.10,131563.81,0.00,0.00,0.00,131563.81
4,90625.97,165449.80,2718.78,253356.99,70939.96,182417.03,3474610.09,45540.10,136876.93,0.00,0.00,0.00,136876.93
5,93344.75,170413.29,2800.34,260957.69,73068.15,187889.54,3578848.39,45540.10,142349.44,3578848.39,232625.15,0.00,3488572.68


In [41]:
# Calculate investment metrics
import numpy as np

# Extract cash flows for IRR calculation
cash_flows = df_transposed['total_cash_flow_to_equity'].values
initial_investment = df_transposed.loc[0, 'initial_equity_investment']

# Adjust cash flows for IRR calculation (initial investment should be negative)
irr_cash_flows = cash_flows.copy()
irr_cash_flows[0] = -initial_investment + cash_flows[0]  # Net cash flow in year 0

# Calculate Unlevered IRR using numpy
def calculate_irr(cash_flows, max_iter=1000, precision=1e-6):
    # Initial guess for IRR
    rate = 0.1
    
    for i in range(max_iter):
        # Calculate NPV and its derivative
        npv = sum([cf / (1 + rate)**t for t, cf in enumerate(cash_flows)])
        npv_derivative = sum([-t * cf / (1 + rate)**(t+1) for t, cf in enumerate(cash_flows)])
        
        if abs(npv) < precision:
            return rate
        
        if npv_derivative == 0:
            break
            
        # Newton-Raphson method
        rate = rate - npv / npv_derivative
        
        if rate < -1:  # Prevent negative rates below -100%
            rate = -0.99
    
    return rate

unlevered_irr = calculate_irr(irr_cash_flows)

# Total returns to equity
total_cash_flow = sum(cash_flows)
total_returns_to_equity = total_cash_flow

# MOIC (Multiple on Invested Capital)
moic = total_returns_to_equity / initial_investment

# Yield on Cost (NOI in final year / initial investment)
final_noi = df_transposed.loc[5, 'NOI']
yield_on_cost = final_noi / initial_investment

# Create metrics summary
metrics = {
    'Initial Equity Investment': f"${initial_investment:,.2f}",
    'Total Returns to Equity': f"${total_returns_to_equity:,.2f}",
    'Unlevered IRR': f"{unlevered_irr:.2%}",
    'MOIC (Multiple on Invested Capital)': f"{moic:.2f}x",
    'Yield on Cost': f"{yield_on_cost:.2%}"
}

print("Investment Metrics Summary:")
print("=" * 40)
for metric, value in metrics.items():
    print(f"{metric}: {value}")


Investment Metrics Summary:
Initial Equity Investment: $2,277,005.00
Total Returns to Equity: $3,975,855.08
Unlevered IRR: 12.70%
MOIC (Multiple on Invested Capital): 1.75x
Yield on Cost: 8.25%
